Objetivo: Cortar y rasterizar el shapefile de manglares de México para entrenar un clasificador de cobertura para imágenes landsat. 

In [ ]:
import landsat.search as search
import landsat.downloader as downloader
from landsat.image import Simple, PanSharpen, FileDoesNotExist
import rasterio
import subprocess
import numpy
import glob
import os
import shutil

In [ ]:
scenes = search.Search().search('031,045', limit = 1, cloud_max = 3, start_date = "2012", end_date = "2016")

In [ ]:
scenes_ids = [str(scene['sceneID']) for scene in scenes['results']]
print scenes_ids

In [ ]:
download = downloader.Downloader()

In [ ]:
for s_id in scenes_ids:
    output = download.download([s_id])

In [ ]:
landsat_dir = "/Users/RaulSierra/landsat/downloads/"
landsat_files = glob.glob(landsat_dir + "LC*.tar.bz")

for file in landsat_files[:2]:
    im_process = Simple(file, bands=[4,3,2,1,5,6,7,9], dst_path="Darwin-SFL-101/landsat_data/", verbose=False)
    im_process.run()

In [ ]:
landsat_data_dir = "Darwin-SFL-101/landsat_data/"
for (root, dirs, files) in os.walk(landsat_data_dir):
    for file in files:
        if file.endswith(".TIF"):
            shutil.move(os.path.join(root,file), landsat_data_dir)
            
    if not dirs:
        shutil.rmtree(root)

## Crear raster de entrenamiento

In [ ]:
!mapshaper Darwin-SFL-101/mapa_manglar/mexoc2010gw/mexoc2010gw.shp -filter-fields Clase -o manglar_clase.shp

Ahora hay que recortar el shape a solo la escena que vamos a usar

In [ ]:
bbox = "-107.31526908421758,20.59854358911711,-105.07951516054297,22.730735318220372"
!mapshaper manglar_clase.shp -clip bbox=$bbox -o manglar_clase_clip.shp 

Utilizamos rasterio para crear un raster en ceros donde rasterizemosel shapefile

In [ ]:
landsat_scene = "/Users/RaulSierra/Ecoinformatica/Devel/iPyNotebooks/Darwin/Darwin-SFL-101/train/landsat/LC80310452014130LGN00_bands_4321567891011.tif"
gt_raster = landsat_scene.replace(".tif", ".gt")

with rasterio.drivers():
    # Read raster bands directly to Numpy arrays.
    #
    with rasterio.open(landsat_scene) as src:
        width = src.width
        height = src.height
        empty_rast = numpy.zeros((width, height), dtype = numpy.uint8)

        
        # Write the product as a raster band to a new 8-bit file. For
        # the new file's profile, we start with the meta attributes of
        # the source file, but then change the band count to 1, set the
        # dtype to uint8, and specify LZW compression.
        profile = src.profile
        profile.update(
            dtype=rasterio.uint8,
            count=1,
            compress='lzw',
            photometric = 'MINISBLACK')

        with rasterio.open(gt_raster, 'w', **profile) as dst:
            dst.write(empty_rast.astype(rasterio.uint8), 1)

    # At the end of the ``with rasterio.drivers()`` block, context
    # manager exits and all drivers are de-registered.

In [ ]:
!rio info $landsat_scene

In [ ]:
!ogr2ogr -t_srs EPSG:3857 manglar_clase_clip_new.shp manglar_clase_clip.shp
!gdal_rasterize -a Clase -l manglar_clase_clip_new manglar_clase_clip_new.shp $gt_raster

In [ ]:
!gdal_calc.py -A $gt_raster -B $landsat_scene --outfile=$gt_raster --calc="A*(B > 0)"

Copiamos el groundtruth para todas las imágenes

In [ ]:
ls_files = glob.glob(landsat_data_dir + "*.TIF")
for ls_file in ls_files:
    shutil.copy(gt_raster, ls_file.replace(".TIF", ".gt"))

Limpiar set de entrenamiento

In [ ]:
tiles_dir = "./Darwin-SFL-101/train/"
gt_tiles = glob.glob(tiles_dir + "*.gt")

with rasterio.drivers():
    # Read raster bands directly to Numpy arrays.
    #
    for tile in gt_tiles:
        with rasterio.open(tile) as src:
            data = src.read()
            if(data.sum() == 0):
                os.remove(tile)
                os.remove(tile.replace(".gt", ".tif"))                
 

In [ ]:
rapideye_name = "1349112_2014-05-17_RE4_3A_255761"
!rio clip $landsat_scene {"LS_" + rapideye_name  + ".tif"} --like {train_dir + rapideye_name + ".tif"}

In [ ]:
!rio warp {train_dir + rapideye_name + ".gt"} {train_dir + "landsat/LS_" + rapideye_name  + ".gt"} --like {train_dir + "landsat/LS_" + rapideye_name  + ".tif"}

In [ ]:
!rio info {train_dir + "landsat/LS_" + rapideye_name  + ".tif"}